In [1]:
"""
Purpose: To demonstrate how to use the 
skeletonization script

"""

'\nPurpose: To demonstrate how to use the \nskeletonization script\n\n'

# Helper Functions

In [145]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np


def filter_mesh_significant_outside_pieces(unfiltered_mesh,main_mesh,
                                           significance_threshold=2000,
                                           n_sample_points=3,
                                          print_flag=False):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces that are not inside of main mesh

    """
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []



    final_mesh_pieces = []

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        #get a random sample of points
        # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
        idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
        points = mesh.vertices[idx,:]


        start_time = time.time()
        signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
        #print(f"Total time = {time.time() - start_time}")

        outside_percentage = sum(signed_distance <= 0)/n_sample_points
        if outside_percentage > 0.9:
            final_mesh_pieces.append(mesh)
            #print(f"Mesh piece {i} OUTSIDE mesh")
        else:
            #print(f"Mesh piece {i} inside mesh :( ")
            pass
                
    return final_mesh_pieces
    
def neuron_boolean_difference(main_mesh_verts,
                             main_mesh_faces,
                             child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=5,
                             significance_threshold=1000,
                             n_sample_points=3,
                             print_flag=False,
                             return_verts_faces=False):
    """
    returns the boolean difference of two meshes passed. Only returns meshes pieces that 
    are greater than the size threshold and outside of the main mesh
    
    Function operation: child_mesh - main_mesh
  

    Parameters: 
    main_mesh_verts (np.array): array of the reference mesh vertices
    main_mesh_faces (np.array): array of the reference mesh faces
    child_mesh_verts (np.array): array of the child mesh vertices that is to be compared to the main mesh
    child_mesh_faces (np.array): array of the child mesh faces that is to be compared to the main mesh
    
    Optional parameters:
    -- for controlling the mesh returned --
    distance_threshold (int) : distance away from the reference mesh that vertices from the child
                                mesh will considered distinct and part of the difference mesh 
                                (default=5)
    significance_threshold (int) : number of faces necessary for any distinct/unconnected part of the 
                                    difference mesh to be considered relevant and included in the difference mesh
                                (default=1000)
    n_sample_points (int) : number of vertices to check to see if submesh is a mesh located inside of the main mesh.
                            The less there are the quicker the speed for filtering the difference mesh
                            (default=3)
    
    
    Returns:
    difference_mesh_verts (np.array): array of vertices from the mesh boolean operation of child - main mesh
    difference_mesh_faces (np.array): array of faces from the mesh boolean operation of child - main mesh
    """
    
    #Create the kdtree from main mesh and run the queries
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(main_mesh_verts)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    pymesh_mesh = trimesh.Trimesh(main_mesh_verts,main_mesh_faces)
    #return new_submesh
    #filter the mesh for only significant pieces on the outside
    returned_mesh = filter_mesh_significant_outside_pieces(new_submesh,pymesh_mesh,
                                                           significance_threshold,
                                                           n_sample_points=n_sample_points,
                                                          print_flag=print_flag)
    
    if return_verts_faces == False:
        return returned_mesh
    else:
        total_returned_mesh = trimesh.Trimesh(vertices = np.array([]),
                                         faces = np.array([]))
        for r in returned_mesh:
            total_returned_mesh = total_returned_mesh + r

        if print_flag:
            print(f"Total time for filtering: {time.time() - start_time}")

        if print_flag:
            print(f"Total time for boolean difference: {time.time() - global_time}")
        return total_returned_mesh.vertices,total_returned_mesh.faces

    
# find mesh that is certain distance away from skeleton: 

def filter_distance_from_skeleton( skeleton_vertices,
                            child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=500,
                             significance_threshold=500,
                             n_sample_points=3, #currently this does not do anything
                             print_flag=False,
                             return_verts_faces=False):
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(skeleton_vertices)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    #gets the faces after has the vertices
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = new_submesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []
    
    #arrange the significant pieces from largest to smallest
    x = [len(k.vertices) for k in significant_pieces]
    sorted_indexes = sorted(range(len(x)), key=lambda k: x[k])
    sorted_indexes = sorted_indexes[::-1]
    sorted_significant_pieces = [significant_pieces[k] for k in sorted_indexes]
    
    return sorted_significant_pieces




In [137]:
main_mesh.vertices.shape,len(main_mesh.vertices)

((517631, 3), 517631)

In [122]:
# x = [70,80,50,100,120]
# sorted_indexes = sorted(range(len(x)), key=lambda k: x[k])
# sorted_indexes[::-1]

[4, 3, 1, 0, 2]

In [26]:
import trimesh
play_mesh_data = np.load("apial_mesh_skeleton.npz",allow_pickle=True)
saved_mesh = play_mesh_data["mesh"]
saved_mesh_object = saved_mesh[()]
skeleton_edges = play_mesh_data["skeleton_edges"]
skeleton_vertices = play_mesh_data["skeleton_vertices"]

In [37]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron

current_mesh = saved_mesh_object
original_mesh_verts = current_mesh.vertices
original_mesh_faces = current_mesh.faces
edges = skeleton_edges
unique_skeleton_verts = skeleton_vertices


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.2]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [54]:
#skeleton_vertices = np.vstack([my_skeleton[:,0],my_skeleton[:,1][-1]])

mesh_pieces = filter_distance_from_skeleton( skeleton_vertices,
                            current_mesh.vertices,
                             current_mesh.faces,
                             distance_threshold=1000,
                             significance_threshold=300,
                             n_sample_points=3,
                             print_flag=False,
                             return_verts_faces=False)

In [55]:
mesh_pieces

[<trimesh.Trimesh(vertices.shape=(20940, 3), faces.shape=(41864, 3))>,
 <trimesh.Trimesh(vertices.shape=(1486, 3), faces.shape=(2964, 3))>]

In [57]:
ipv.figure()
for mi in mesh_pieces:
    ipv.plot_trisurf(mi.vertices[:,0], 
                            mi.vertices[:,1],
                            mi.vertices[:,2], 
                            triangles=mi.faces, color='orange')

    mesh.color = [0., 1., 0., 0.2]
    mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [85]:
import os, contextlib
import pathlib
import subprocess

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [86]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="None",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        if folder == "None":
            file_loc = pathlib.Path.cwd()
            
        else:
            file_loc = pathlib.Path.cwd() / folder
            
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

In [87]:
"""
Pseudocode for new surface skeletonization integration:

Changes in parameters: 
1) Have skeletonization type
2) Need to lower the significance thresholds so picks up more pieces
3) Set max_depth
4) current_depth
5) surface_samples

Process: 
1) Pass through skeletonization type:
2) If cgal type --> do everything the same:
3) if no significant poisson piece --> do surface_with_poisson process
    change the name slightly
    change the skeletonization type
    make recursive call
4) if too deep in layers --> do surface_without_poisson
    change the name slightly
    change the skeletonization type
    make recursive call
    
5) If works --> then do cgal and read in skeleton 


surface_with_poisson:
1) same all the way but not do cgal skeletonization and instead do skeletonization on the mesh poisson largest mesh itself
2) read in the skeleton (be ready to pash it to boolean mesh subtraction)
3) Write the skeleton as a cgal file

surface_without_poisson:
1) just do surface skeletonization on the mesh
2) read in the skeleton (be ready to pash it to boolean mesh subtraction)
3) write the skeleton as cgal file




"""

'\nPseudocode for new surface skeletonization integration:\n\nChanges in parameters: \n1) Have skeletonization type\n2) Need to lower the significance thresholds so picks up more pieces\n3) Set max_depth\n4) current_depth\n5) surface_samples\n\nProcess: \n1) Pass through skeletonization type:\n2) If cgal type --> do everything the same:\n3) if no significant poisson piece --> do surface_with_poisson process\n    change the name slightly\n    change the skeletonization type\n    make recursive call\n4) if too deep in layers --> do surface_without_poisson\n    change the name slightly\n    change the skeletonization type\n    make recursive call\n    \n5) If works --> then do cgal and read in skeleton \n\n\nsurface_with_poisson:\n1) same all the way but not do cgal skeletonization and instead do skeletonization on the mesh poisson largest mesh itself\n2) read in the skeleton (be ready to pash it to boolean mesh subtraction)\n3) Write the skeleton as a cgal file\n\nsurface_without_poisson

In [124]:
"""
Extra skeleton functions:
"""
import networkx as nx
import time 
import numpy as np
import trimesh
import random

def generate_surface_skeleton(vertices, faces, surface_samples):
    
    #return surface_with_poisson_skeleton,path_length
    
    current_mesh = trimesh.Trimesh(vertices=vertices,
                                  faces = faces)


    start_time = time.time()

    # test on a sphere mesh
    mesh = current_mesh

    ga = nx.from_edgelist(mesh.edges)

    if surface_samples<len(vertices):
        k = surface_samples
    else:
        k = len(vertices)
    sampled_nodes = random.sample(ga.nodes, k)


    lp_end_list = []
    lp_magnitude_list = []

    for s in sampled_nodes: 
        sp_dict = nx.single_source_shortest_path_length(ga,s)

        list_keys = list(sp_dict.keys())
        longest_path_node = list_keys[len(list_keys)-1]
        longest_path_magnitude = sp_dict[longest_path_node]


        lp_end_list.append(longest_path_node)
        lp_magnitude_list.append(longest_path_magnitude)

    #construct skeleton from shortest path
    final_start = sampled_nodes[np.argmax(lp_magnitude_list)]
    final_end = sampled_nodes[np.argmax(lp_end_list)]

    node_list = nx.shortest_path(ga,final_start,final_end)
    if len(node_list) < 2:
        print("node_list len < 2 so returning empty list")
        return np.array([])
    #print("node_list = " + str(node_list))
    final_skeleton = mesh.vertices[np.vstack([node_list[:-1],node_list[1:]]).T]
    print(f"   Final Time for surface skeleton with sample size = {k} = {time.time() - start_time}")

    return final_skeleton

def save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path):
    """
    surface_with_poisson_skeleton (np.array) : nx2 matrix with the nodes
    """
    first_node = surface_with_poisson_skeleton[0][0]
    end_nodes =  surface_with_poisson_skeleton[:,1]
    
    skeleton_to_write = str(len(end_nodes) + 1) + " " + str(first_node[0]) + " " +  str(first_node[1]) + " " +  str(first_node[2])
    
    for node in end_nodes:
        skeleton_to_write +=  " " + str(node[0]) + " " +  str(node[1]) + " " +  str(node[2])
    
    output_file = largest_mesh_path
    if output_file[-5:] != ".cgal":
        output_file += ".cgal"
        
    f = open(output_file,"w")
    f.write(skeleton_to_write)
    f.close()
    return 

#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)
    unique_skeleton_verts = bone_array_total.reshape(-1,3)

def calculate_skeleton_distance(my_skeleton):
    total_distance = np.sum(np.sqrt(np.sum((my_skeleton[:,0] - my_skeleton[:,1])**2,axis=1)))
    return float(total_distance)



"""
Testing if the read and write are correct:

import numpy as np
file_name = "107816118160698192_leftover_1-0_skeleton.cgal"
my_skeleton = read_skeleton_revised(file_name)
my_skeleton
practice_file = "test_manual_cgal.cgal"
save_skeleton_cgal(my_skeleton,practice_file)
my_skeleton_2 = read_skeleton_revised(practice_file)

"""

'\nTesting if the read and write are correct:\n\nimport numpy as np\nfile_name = "107816118160698192_leftover_1-0_skeleton.cgal"\nmy_skeleton = read_skeleton_revised(file_name)\nmy_skeleton\npractice_file = "test_manual_cgal.cgal"\nsave_skeleton_cgal(my_skeleton,practice_file)\nmy_skeleton_2 = read_skeleton_revised(practice_file)\n\n'

In [172]:
def recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth,
                            skeletonization_type,
                            surface_samples=200,
                            max_depth_cgal = 8,
                            max_depth_surface = 10,
                            min_skeleton_distance = 30,
                           boolean_distance_threshold=1000,
                            boolean_significance_threshold=1000,
                            boolean_n_sample_points=10,
                            boolean_print_flag = False,
                           largest_mesh_significant=3000):
    

    print(f"\n----------Starting {current_name}----------")
    global_time = time.time()
    #print out the off file
    
    if current_depth >= max_depth_surface:
        print(f"\n\n*********Reamched maximum depth of {max_depth_surface} so returning********\n\n")
    if skeletonization_type== "cgal" and current_depth >= max_depth_cgal:
        print("     Reached max depth cgal")

        skeletonization_type = "surface_with_poisson"
        current_name += "_sWp"

        recursive_skeletonizing(verts,faces,current_name,
                        directory,current_base_path,mesh_base_path,folder_name,
                        current_depth=current_depth,
                        skeletonization_type=skeletonization_type,
                        surface_samples=surface_samples,
                        max_depth_cgal = max_depth_cgal,
                        max_depth_surface = max_depth_surface,
                        min_skeleton_distance = min_skeleton_distance,
                       boolean_distance_threshold=boolean_distance_threshold,
                        boolean_significance_threshold=boolean_significance_threshold,
                        boolean_n_sample_points=boolean_n_sample_points,
                        boolean_print_flag = boolean_print_flag,
                       largest_mesh_significant=largest_mesh_significant)

    
    
    
    if skeletonization_type == "surface_without_poisson":
        surface_without_poisson_skeleton = generate_surface_skeleton(verts,
                                                                      faces=faces,
                                                                      surface_samples = surface_samples)
        if len(surface_without_poisson_skeleton) <= 0 :
            print("Returning because the skeleton returned was of size 0")
            return
        #print(f"Path length for surface skeleton WITHOUT poisson = {path_length}")

        largest_mesh_path = mesh_base_path + current_name

        #save off the skeleton
        save_skeleton_cgal(surface_without_poisson_skeleton,largest_mesh_path + "_skeleton")
        
        
    else:
        
        initial_output_path = mesh_base_path + current_name

        write_Whole_Neuron_Off_file(vertices=verts, 
                                    triangles=faces,
                                   path_and_filename=initial_output_path)


        # do the poisson surface reconstruction

        script_name = "poisson_working_meshlab.mls"

        input_file_base = initial_output_path
        output_file = input_file_base + "_poisson"
        meshlab_script_path_and_name =current_base_path + script_name

        start_time = time.time()
        print("     Starting Screened Poisson")
        meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                           output_path_and_filename=output_file + ".off",
                                                         meshlab_script=meshlab_script_path_and_name)
        print(f"     Total_time for Screended Poisson = {time.time() - start_time}")

        #2) Filter away for largest_poisson_piece:

        new_mesh = trimesh.load_mesh(output_file + ".off")
        mesh_splits = new_mesh.split(only_watertight=False)
        mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
        largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
        largest_mesh = mesh_splits[largest_index][0]

        print("len(largest_mesh.vertices) = " + str(len(largest_mesh.vertices)))
        #4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
        if len(largest_mesh.vertices) < largest_mesh_significant:
            print(current_name + " largest mesh not significant AFTER POISSON RECONSTRUCTION")
            print("trying surface skeletonization without Poisson")
            
            """3) if no significant poisson piece --> do surface_with_poisson process
                change the name slightly
                change the skeletonization type"""
            
            skeletonization_type = "surface_without_poisson"
            current_name += "_sWOp"
            
            recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=current_depth,
                            skeletonization_type=skeletonization_type,
                            surface_samples=surface_samples,
                            max_depth_cgal = max_depth_cgal,
                            max_depth_surface = max_depth_surface,
                            min_skeleton_distance = min_skeleton_distance,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=boolean_n_sample_points,
                            boolean_print_flag = boolean_print_flag,
                           largest_mesh_significant=largest_mesh_significant)
            return 
         
        if skeletonization_type == "surface_with_poisson":
            surface_with_poisson_skeleton = generate_surface_skeleton(largest_mesh.vertices,
                                                                      faces=largest_mesh.faces,
                                                                      surface_samples = surface_samples)
            
            if len(surface_with_poisson_skeleton) <= 0 :
                print("Returning because the skeleton returned was of size 0")
                return
            #print(f"Path length for surface skeleton with poisson = {path_length}")
            
            largest_mesh_path = mesh_base_path + current_name
            
            #save off the skeleton
            save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path + "_skeleton")


        elif skeletonization_type == "cgal":

            #5) If significant size output the mesh
            largest_mesh_path = mesh_base_path + current_name
            write_Whole_Neuron_Off_file(vertices=largest_mesh.vertices, 
                                        triangles=largest_mesh.faces,
                                       path_and_filename=largest_mesh_path)


            #6) Run skeletonization on it:
            start_time = time.time()
            print("     Starting Calcification")
            cm.calcification(largest_mesh_path)
            print(f"     Total_time for Calcification = {time.time() - start_time}")




    """ ***** GOING TO USE THE SKELETON VERTICES INSTEAD OF THE MESH VERTICES"""
        

    #7) Subtract skeleton vertices from current mesh
    start_time = time.time()
    print("     Starting Mesh boolean difference")

    cgal_skeleton_file = largest_mesh_path + "_skeleton.cgal"
    
    try:
        returned_skeleton = read_skeleton_revised(cgal_skeleton_file)
    except OSError as e:
        print("\n\n**** No cgal file was found so returning ******\n\n")
        return
    else:
        pass
        #print("\n\n**** unknown error ocured when reading in cgal file so returning ******\n\n")
        #return
        

    skeletal_length = calculate_skeleton_distance(returned_skeleton)
    print(f" Skeletal Length = {skeletal_length}  ")
    # returning based on skeletal length: 
    if skeletal_length <= min_skeleton_distance:
        
        if skeletonization_type == "cgal":
            print("Skeleton generated was too small --> cgal skeletonization turning into surface with poisson")
            #try the surface skeletonization with poisson
            current_name += "sWp"
            skeletonization_type = "surface_with_poisson"
         
        elif skeletonization_type == "surface_with_poisson":
            print("Skeleton generated was too small --> surface WITH poisson turning into surface WTIHOUT poisson")
            current_name += "sWOp"
            skeletonization_type = "surface_without_poisson"
        else:
            return
            
        recursive_skeletonizing(verts,faces,current_name,
                    directory,current_base_path,mesh_base_path,folder_name,
                    current_depth=current_depth,
                    skeletonization_type=skeletonization_type,
                    surface_samples=surface_samples,
                    max_depth_cgal = max_depth_cgal,
                    max_depth_surface = max_depth_surface,
                    min_skeleton_distance = min_skeleton_distance,
                   boolean_distance_threshold=boolean_distance_threshold,
                    boolean_significance_threshold=boolean_significance_threshold,
                    boolean_n_sample_points=boolean_n_sample_points,
                    boolean_print_flag = boolean_print_flag,
                   largest_mesh_significant=largest_mesh_significant)
        return
            
        


    skeleton_vertices = np.vstack([returned_skeleton[:,0],returned_skeleton[:,1]])
    #getting the boolean difference
#     mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
#                                    largest_mesh.faces,
#                                     verts,
#                                    faces,
#                                    distance_threshold=boolean_distance_threshold,
#                                    significance_threshold=boolean_significance_threshold,
#                                    n_sample_points=boolean_n_sample_points,
#                                   print_flag = boolean_print_flag)

#     mesh_pieces =neuron_boolean_difference( skeleton_vertices
#                                 verts,
#                                faces,
#                                distance_threshold=boolean_distance_threshold,
#                                significance_threshold=boolean_significance_threshold,
#                                n_sample_points=boolean_n_sample_points,
#                               print_flag = boolean_print_flag)
    
    if skeletonization_type == "cgal":
        current_boolean_distance_threshold = boolean_distance_threshold
    else:
        current_boolean_distance_threshold = 500
    
    mesh_pieces = filter_distance_from_skeleton( skeleton_vertices,
                            verts,
                             faces,
                             distance_threshold=current_boolean_distance_threshold,
                             significance_threshold=boolean_significance_threshold,
                             n_sample_points=boolean_n_sample_points,
                             print_flag=boolean_print_flag,
                             return_verts_faces=False)



    print(f"     Total_time for Mesh boolean difference = {time.time() - start_time}")

    print(f"Total time for one mesh piece skeleton = {time.time() - global_time}")

    print(f"{current_name} there were {len(mesh_pieces)} significant pieces leftover after largest mesh")

    if len(mesh_pieces) <= 0:
        print(f"{current_name} returning because 0 significant pieces")
        return 

    
    
    #increment the current depth 
    current_depth = current_depth + 1

    print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
    # check if the number of 
    for j,piece in enumerate(mesh_pieces):
        new_current_name = current_name + "-" + str(j)
        print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
        recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                               directory=directory,current_base_path=current_base_path,
                                mesh_base_path=mesh_base_path,folder_name=folder_name,
                               current_depth=current_depth,
                                skeletonization_type=skeletonization_type,
                                surface_samples=surface_samples,
                                max_depth_cgal = max_depth_cgal,
                                max_depth_surface = max_depth_surface,
                                min_skeleton_distance = min_skeleton_distance,
                               boolean_distance_threshold=boolean_distance_threshold,
                                boolean_significance_threshold=boolean_significance_threshold,
                                boolean_n_sample_points=boolean_n_sample_points,
                                boolean_print_flag = boolean_print_flag,
                               largest_mesh_significant=largest_mesh_significant)
    
    return

In [140]:
# #read in the skeleton files into an array
# def read_skeleton_revised(file_path):
#     with open(file_path) as f:
#         bones = np.array([])
#         for line in f.readlines():
#             #print(line)
#             line = (np.array(line.split()[1:], float).reshape(-1, 3))
#             #print(line[:-1])
#             #print(line[1:])

#             #print(bones.size)
#             if bones.size <= 0:
#                 bones = np.stack((line[:-1],line[1:]),axis=1)
#             else:
#                 bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
#             #print(bones)


#     return np.array(bones).astype(float)

In [141]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

import matplotlib.pyplot as plt #for graph visualization
from scipy.spatial import distance_matrix
from tqdm import tqdm

# reimplementation of connected_component_subgraphs deprecated in networkx>=2.4
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

def stitch_skeleton_with_degree_check_vp2(staring_edges,max_stitch_distance=18000,end_node=False):
    

    #unpacks so just list of vertices
    vertices_unpacked  = staring_edges.reshape(-1,3)

    #reduce the number of repeat vertices and convert to list
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()

    #assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
    vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]

    #reshapes the vertex list to become an edge list
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    
    #create the graph from the edges
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)

    # find the shortest distance between the two different subgraphs:
    from scipy.spatial import distance_matrix

    UG = B.to_undirected()

    # extract subgraphs
    sub_graphs = connected_component_subgraphs(UG)

    subgraphs_list = []
    for i, sg in enumerate(sub_graphs):
        #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
        #print("\tNodes:", sg.nodes(data=True))
        #print("\tEdges:", sg.edges())
        subgraphs_list.append(sg)

    print("len_subgraphs AT BEGINNING = " + str(len(subgraphs_list)))

    while len(subgraphs_list) > 1:
#         if subgraph_counter>=len(subgraphs_list):
#             print("Breaking because the sugraph counter was larger than list")
#             break
#         print("subgraph_counter = " + str(subgraph_counter))
        current_sub_graph = subgraphs_list[0]
        #get all of the coordinates of this subgraph 
        #get all of the coordinates
        if end_node == True:
            current_nodes = np.array([x for x,n in current_sub_graph.degree() if n <= 1])
            current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
            current_coord_filter = { k:v for k,v in current_coord.items() if k in  current_nodes}
            current_coordinates = np.array(list(current_coord_filter.values()))
        
        else:
            #get all of the coordinates
            current_nodes = np.array(list(current_sub_graph.nodes))
            current_coord = nx.get_node_attributes(current_sub_graph,'coordinates')
            current_coordinates = np.array(list(current_coord.values()))
            
#         coord = nx.get_node_attributes(current_sub_graph,'coordinates')
#         current_nodes = np.array(list(current_sub_graph.nodes))
#         current_coordinates = np.array(list(coord.values()))

        #store the minimum distance
        min_dist = np.inf
        min_dist_subgraph_index = -1
        min_dist_edge = [-1,-1]
        min_dist_edge_index = [-1,-1]

        #i = 0
        for i, sg in enumerate(subgraphs_list):
            if i == 0:
                continue

            #get all of the coordinates
            if end_node == True:
                new_nodes = np.array([x for x,n in sg.degree() if n <= 1])
                #print("new nodes = " + str(new_nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                #print("new_coord = " + str(new_coord))
                new_coord_filter = { k:v for k,v in new_coord.items() if k in  new_nodes}
                new_coordinates = np.array(list(new_coord_filter.values()))
            else:
                #get all of the coordinates
                new_nodes = np.array(list(sg.nodes))
                new_coord = nx.get_node_attributes(sg,'coordinates')
                new_coordinates = np.array(list(new_coord.values()))
                
            #find the closest distance between those points
            a_New = current_coordinates
            b_New = new_coordinates

            # print("starting distance matrix")
            start_time = time.time()

            a_b_distance = distance_matrix(a_New, b_New)
            #print(a_b_distance)

            current_min = np.min(a_b_distance)
            dist_matrix_index = np.where(a_b_distance == current_min)
            min_indexes = dist_matrix_index[0][0],dist_matrix_index[1][0]

            #print("Min indexes = " + str(min_indexes))

            if current_min<min_dist:
                min_dist = current_min
                min_dist_subgraph_index = i
                min_dist_edge_index = [current_nodes[min_indexes[0]],
                                       new_nodes[min_indexes[1]]]
                min_dist_edge = [current_coordinates[min_indexes[0]],
                                        new_coordinates[min_indexes[1]]]
            #i += 1

        #add the shortest connection to the overall edges
        print("min_dist = " + str(min_dist))
        print("min_dist_subgraph_index = " + str(min_dist_subgraph_index))
        print("min_dist_edge_index = " + str(min_dist_edge_index))
        print("min_dist_edge = " + str(min_dist_edge))
        
        #check if the distance is too far:
        if min_dist > max_stitch_distance:
            print("********** breaking because min distance is too large*****")
            break
           
        

        #adds the new edge to the graph
        B.add_edge(*min_dist_edge_index)

        #
        #print("edges after adding")
        #print(B.edges())
        UG = B.to_undirected()

        # extract subgraphs
        sub_graphs = connected_component_subgraphs(UG)
        subgraphs_list = []
        for i, sg in enumerate(sub_graphs):
#             print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#             print("\tNodes:", sg.nodes(data=True))
#             print("\tEdges:", sg.edges())
            subgraphs_list.append(sg)

        print("len_subgraphs AT END= " + str(len(subgraphs_list)))
        
        
    # get the largest subgraph!!! in case have missing pieces
    
    #add all the new edges to the 
    Gc = max(connected_component_subgraphs(B), key=len)
    
    total_coord = nx.get_node_attributes(B,'coordinates')
    #print("len(total_coord)= " + str(len(total_coord)))
    current_coordinates = np.array(list(total_coord.values()))
    #print("current_coordinates.shape= " + str(current_coordinates.shape))
    #print("Gc.edges() = " + str(Gc.edges()))
    return current_coordinates[Gc.edges()]


"""
For finding the distances of the skeleton
"""
def find_skeleton_distance(example_edges):
    total_distance = np.sum([np.linalg.norm(a-b) for a,b in example_edges])
    return total_distance

from scipy.spatial import distance

def find_skeleton_distance_scipy(example_edges):
    total_distance = np.sum([distance.euclidean(a, b) for a,b in example_edges])
    return total_distance

# Main Function 

In [169]:
import calcification_Module as cm
import time
import os
import pathlib

def calculate_skeleton_edges(input_verts,input_faces,segment_id,
                            boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   surface_samples=200,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300):
    original_mesh = trimesh.Trimesh(vertices=input_verts,faces=input_faces)
    name2 = segment_id
    
    
    directory = "./" + name2
    if not os.path.exists(directory):
        os.makedirs(directory)


    current_base_path = str(pathlib.Path.cwd()) + "/"
    mesh_base_path = current_base_path + str(name2) + "/"
    folder_name = str(name2)

    current_layer = 1
    
#     recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
#                             directory,current_base_path,mesh_base_path,folder_name,
#                         boolean_distance_threshold=boolean_distance_threshold,
#                             boolean_significance_threshold=boolean_significance_threshold,
#                             boolean_n_sample_points=boolean_n_sample_points,
#                             boolean_print_flag = boolean_print_flag,
#                            largest_mesh_significant=largest_mesh_significant)
    
    
    recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=1,
                            skeletonization_type="cgal",
                            surface_samples=surface_samples,
                            max_depth_cgal = 8,
                            max_depth_surface = 10,
                            min_skeleton_distance = 500,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=surface_samples,
                            boolean_print_flag = False
                           )
    
    
    
    
    cgal_skeleton_file_list = []
    
    for file in os.listdir(name2):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")
    
    bone_array_total = np.vstack([read_skeleton_revised(k) 
                                  for k in cgal_skeleton_file_list])

    unique_skeleton_verts = bone_array_total.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


    total_edges = np.array([])

    for k in cgal_skeleton_file_list:
        bone_array = read_skeleton_revised(k) 

        #add the skeleton edges to the total edges
        if not total_edges.any():
            total_edges = bone_array
        else:
            total_edges = np.vstack([total_edges,bone_array])

    total_edges_stitched = stitch_skeleton_with_degree_check_vp2(total_edges)
    
    unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)
    
#     #erase the folder with all of the data in it
#     import shutil
#     shutil.rmtree(directory)
    
    #return the edges
    return edges,unique_skeleton_verts


    


In [170]:
# Calling the actual function
#main_mesh = trimesh.load_mesh("Surfae_Skeleton/107816118160698192_original_decimated.off")
main_mesh = trimesh.load_mesh("L5_neuron.off")
segment_id = "Layer_5"

In [176]:
import time
total_start_time = time.time()

edges,unique_skeleton_verts = calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,segment_id,
                                   boolean_distance_threshold=2000,
                                   boolean_significance_threshold=1500,
                                   surface_samples=200,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=500)
print(f"Total time for calculating skeleton = {time.time() - start_time}")


----------Starting Layer_5----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_Decimation_2_26/Layer_5/Layer_5.off -o /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_Decimation_2_26/Layer_5/Layer_5_poisson.off -s /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_Decimation_2_26/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 41.33542442321777


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

len(largest_mesh.vertices) = 32460
     Starting Calcification
     Total_time for Calcification = 1.768662691116333
     Starting Mesh boolean difference
 Skeletal Length = 791605.7817829953  
     Total_time for Mesh boolean difference = 6.171024560928345
Total time for one mesh piece skeleton = 61.03136205673218
Layer_5 there were 71 significant pieces leftover after largest mesh
 -->  Total significant pieces left: [30853, 20128, 18966, 14389, 13855, 13324, 13313, 12893, 12125, 11461, 10674, 10331, 10127, 9245, 8961, 8906, 8457, 8365, 8279, 8126, 7727, 7287, 7253, 6231, 6039, 5596, 4934, 4792, 4592, 4487, 4080, 4072, 3968, 3705, 3482, 3481, 3101, 3067, 2874, 2533, 2395, 2392, 2223, 2196, 2065, 2032, 2009, 1988, 1683, 1537, 1517, 1408, 1272, 1187, 1087, 1072, 1058, 1015, 1004, 996, 961, 941, 886, 880, 878, 851, 824, 818, 770, 770, 754]
Starting recursive call for size vertices of 30853

----------Starting Layer_5-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 8

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

len(largest_mesh.vertices) = 83504
     Starting Calcification
     Total_time for Calcification = 4.455974102020264
     Starting Mesh boolean difference
 Skeletal Length = 223151.79127114118  
THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold
     Total_time for Mesh boolean difference = 0.31896233558654785
Total time for one mesh piece skeleton = 20.452561378479004
Layer_5-0 there were 0 significant pieces leftover after largest mesh
Layer_5-0 returning because 0 significant pieces
Starting recursive call for size vertices of 20128

----------Starting Layer_5-1----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_Decimation_2_26/Layer_5/Layer_5-1.off -o /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_Decimation_2_26/Layer_5/Layer_5-1_poisson.off -s /notebooks3/U

...

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


len(largest_mesh.vertices) = 38446
node_list = [22022, 21993, 27961, 26750, 33162, 26687, 30710, 30737, 6332, 34641, 5814, 10191, 14941, 19298, 22899, 27744, 27810, 31747, 35950, 1650, 11022, 23999, 32491, 5665, 2813, 12190, 20772, 29468, 9221, 3828, 13169, 21377, 30341, 114, 9543, 18367, 918, 31237, 1155]
   Final Time for surface skeleton with sample size = 200 = 50.59263896942139
     Starting Mesh boolean difference
 Skeletal Length = 4105.477324121335  
     Total_time for Mesh boolean difference = 0.15096569061279297
Total time for one mesh piece skeleton = 59.54154300689697
Layer_5-7-0-0-0-0-0-0_sWp-0-0-0 there were 1 significant pieces leftover after largest mesh
 -->  Total significant pieces left: [10361]
Starting recursive call for size vertices of 10361

----------Starting Layer_5-7-0-0-0-0-0-0_sWp-0-0-0-0----------


*********Reamched maximum depth of 10 so returning********


     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebo

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


len(largest_mesh.vertices) = 37584
node_list = [21859, 21774, 9507, 18233, 9006, 4383, 37496, 33208, 29419, 19473, 20785, 20656, 12598, 36958, 3520, 1548, 32663, 25287, 20058, 11587, 2441, 27290, 31536, 20574, 23276, 10593, 1366, 34758, 30736, 22239, 30324, 14265, 5161, 34035, 21257, 13159, 4089, 24810, 2940, 2955, 23932, 6795, 22940, 5908, 26561, 4724, 6208, 29874, 25445, 3126, 8236, 279, 36680, 28628, 24402, 36513, 16246, 25146, 7227, 35857, 36904, 28067, 15360, 6453, 34984, 34991, 26913, 33738, 18500, 9829, 480, 250, 25851, 17687, 8896, 37049, 4971, 29099, 16651, 7684, 36110, 28280, 19751, 11174, 2107, 27404, 19103, 10156, 10165, 988, 30524, 30391]
   Final Time for surface skeleton with sample size = 200 = 50.36553144454956
     Starting Mesh boolean difference
 Skeletal Length = 15241.333462632456  
     Total_time for Mesh boolean difference = 0.12205719947814941
Total time for one mesh piece skeleton = 59.353440284729004
Layer_5-7-0-0-0-0-0-0_sWp-0-0-0-0 there were 1 significant

KeyboardInterrupt: 

## np.savez("L5_skeleton_try1.npz",mesh=main_mesh,skeleton_edges=edges,unique_skeleton_verts=unique_skeleton_verts)

# Visualizing the unstithed mesh and skeleton

In [174]:
name2 = "Layer_5"
cgal_skeleton_file_list = []

for file in os.listdir(name2):
    if file.endswith(".cgal"):
        cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
if len(cgal_skeleton_file_list) <= 0:
    #check to see if cgal list is empty
    raise Exception("There were no cgal files generated in process")

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts = bone_array_total.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


In [175]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure(figsize=(15,15))
# we draw the tetrahedron

current_mesh = saved_mesh_object
original_mesh_verts = main_mesh.vertices
original_mesh_faces = main_mesh.faces
edges = edges
unique_skeleton_verts = unique_skeleton_verts


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# Visualizing the mesh and the skeleton:

In [158]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure(figsize=(15,15))
# we draw the tetrahedron

current_mesh = saved_mesh_object
original_mesh_verts = main_mesh.vertices
original_mesh_faces = main_mesh.faces
edges = edges
unique_skeleton_verts = unique_skeleton_verts


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [118]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure(figsize=(15,15))
# we draw the tetrahedron

current_mesh = saved_mesh_object
original_mesh_verts = main_mesh.vertices
original_mesh_faces = main_mesh.faces
edges = edges
unique_skeleton_verts = unique_skeleton_verts


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices


volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

# checking the non-skeletonized part

In [ ]:
skeleton_file = "L5-6-0-0-0-0-0_skeleton.cgal"

# Function call to skeletonize a mesh with vertices and faces

In [ ]:
import shutil

from minfig import * # Required for the adapters to be used with locally defined tables

# Virtual module accessors
minnie = configure_minnie(return_virtual_module=True) # virtual module with the adapted attribute for mesh access from .h5 files

@minnie.schema
class Skeleton(dj.Computed):
    definition = minnie.Skeleton.definition
    
    key_source = minnie.Decimation.proj(decimation_version='version') * (minnie.SkeletonVersion.proj() & 'version=0')

    def make(self, key):
        main_mesh = (minnie.Decimation & key).fetch1('mesh')
                
        try:
            edges,unique_skeleton_verts = calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,str(main_mesh.segment_id),
                                   boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   boolean_n_sample_points=10,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300)
            print(f'Inserting {key["segment_id"]} into the table.')
            self.insert1(dict(
                key,
                vertices=unique_skeleton_verts,
                edges=edges
            ))
        except Exception as e:
            raise e
        finally:
            temp_dir = os.path.join('/notebooks/Platinum_Blender', str(main_mesh.segment_id))
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)

if __name__ == '__main__':
    import random
    import time

    # Random sleep delay to avoid concurrent key_source queries from hangin
#     time.sleep(random.randint(0, 360))
    print('Populate Started')
    Skeleton.populate(reserve_jobs=True, suppress_errors=True, order='random')
#     Skeleton.populate(suppress_errors=True)

Connecting celiib@10.28.0.34:3306
Populate Started

----------Starting 89454824134979798----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/89454824134979798/89454824134979798.off -o /notebooks/Platinum_Blender/89454824134979798/89454824134979798_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 6.720912218093872


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


len(largest_mesh.vertices) = 4391
     Starting Calcification
     Total_time for Calcification = 0.1837749481201172
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.29581546783447266
Total time for one mesh piece skeleton = 7.760610818862915
89454824134979798 there were 3 significant pieces leftover after largest mesh

----------Starting 89454824134979798-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/89454824134979798/89454824134979798-0.off -o /notebooks/Platinum_Blender/89454824134979798/89454824134979798-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 5.204952716827393
len(largest_mesh.vertices) = 705
     Starting Calcification
     Total_time for Calcification = 0.0572969913482666
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.0537564754486084
Total time for 

face_normals all zero, ignoring!


     Total_time for Screended Poisson = 5.06619930267334
len(largest_mesh.vertices) = 729
     Starting Calcification
     Total_time for Calcification = 0.0627446174621582
     Starting Mesh boolean difference
THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold
     Total_time for Mesh boolean difference = 0.009842157363891602
Total time for one mesh piece skeleton = 5.181253433227539
89454824134979798-0-0-0 there were 0 significant pieces leftover after largest mesh
89454824134979798-0-0-0 returning because 0 significant pieces

----------Starting 89454824134979798-0-1----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/89454824134979798/89454824134979798-0-1.off -o /notebooks/Platinum_Blender/89454824134979798/89454824134979798-0-1_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 5.03771710395813
len(largest_mesh.vertices) = 1487
   

...

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


     Total_time for Screended Poisson = 5.148973703384399
len(largest_mesh.vertices) = 1292
     Starting Calcification
     Total_time for Calcification = 0.09609389305114746
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.06200361251831055
Total time for one mesh piece skeleton = 5.430055618286133
102677344073980225-6-0 there were 1 significant pieces leftover after largest mesh

----------Starting 102677344073980225-6-0-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/102677344073980225/102677344073980225-6-0-0.off -o /notebooks/Platinum_Blender/102677344073980225/102677344073980225-6-0-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 5.038171768188477
len(largest_mesh.vertices) = 723
     Starting Calcification
     Total_time for Calcification = 0.07646560668945312
     Starting Mesh boolean differenc


----------Starting 100213613058696161----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/100213613058696161/100213613058696161.off -o /notebooks/Platinum_Blender/100213613058696161/100213613058696161_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


     Total_time for Screended Poisson = 6.324129581451416
len(largest_mesh.vertices) = 1395
     Starting Calcification
     Total_time for Calcification = 0.07988476753234863
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 0.13840937614440918
Total time for one mesh piece skeleton = 6.805828332901001
100213613058696161 there were 2 significant pieces leftover after largest mesh

----------Starting 100213613058696161-0----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Blender/100213613058696161/100213613058696161-0.off -o /notebooks/Platinum_Blender/100213613058696161/100213613058696161-0_poisson.off -s /notebooks/Platinum_Blender/poisson_working_meshlab.mls


In [28]:
minnie.schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decimation,9e58782d18190ae8ce872ee478804b3b,error,=BLOB=,OSError: Can't read data (inflate() failed),=BLOB=,cpapadop@10.28.0.175,at-node48,7,193469,2020-01-21 02:22:56


In [ ]:
import trimesh
import shutil

# key = {'segment_id': 81429626202945974}
key = {'segment_id': 107816118160698192}

main_mesh = (minnie.Decimation & key).fetch1('mesh')

try:
    edges,unique_skeleton_verts = calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,str(main_mesh.segment_id),
                                   boolean_distance_threshold=3000,
                                   boolean_significance_threshold=1000,
                                   boolean_n_sample_points=10,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300)
except Exception as e:
    raise e
finally:
    temp_dir = os.path.join('/notebooks/Platinum_Blender', str(main_mesh.segment_id))
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

In [ ]:
# test that the skeletonization went well
import ipyvolume as ipv
ipv.figure(width=1024, height=1024)
# we draw the tetrahedron
original_mesh = main_mesh


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [ ]:
"""
Pseudocode for skeleton stitcher: 
-------Old method-------

1) creates a network with unique nodes and their coordinates and the edges of the graph
2) extract subgraphs: Appends the list of subgraphs
3) Iterates through all of the subgraphs
a. gets first subgraph
b. gets current coordinates of subgraph
c. strip these coordinates from all the list of entire coordinates
d. Find closest node
e. Add edge (make sure not greater than max length)
f. recompute the subgraphs
g. keep looping until only one component
4) Output the edges as a skeleton of coordinates



"""